In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from privacypacking.utils.utils import LOGS_PATH
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve
import yaml

In [ ]:
print(yaml.dump(LaplaceCurve(1/4.9).epsilons))
print()
print(yaml.dump(LaplaceCurve(1/1.9).epsilons))

In [ ]:
# logs = LOGS_PATH.joinpath("ray/run_and_report_2021-10-29_17-22-50")

# df = load_ray_experiment(logs)

df = load_latest_ray_experiment()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
px.line(df.sort_values('n_blocks'), x="n_blocks", y="n_allocated_tasks", color="scheduler", title="Multiblock DPF killer", facet_col="block_selecting_policy")

In [ ]:
# Zeta
alpha = 0.1
n_blocks = 10
density = np.array([1 / (i + 1) ** (alpha) for i in range(n_blocks)])
density /= sum(density)

px.line(density)

In [ ]:
import json
from collections import defaultdict

In [ ]:
p1 = "/home/pierre/privacypacking/logs/basic_scheduler_DominantShares/1029-184048.json"
p2 = "/home/pierre/privacypacking/logs/simplex_DominantShares/1029-184048.json"

with open(p1) as f:
    d1 = json.load(f)
    
df = pd.DataFrame(d1["tasks"])



json_log_paths = [p1,p2]

In [ ]:
list(LOGS_PATH.glob("exp_*"))

In [ ]:
def color_from_row(row):
    return 100* list(row["budget_per_block"].values())[0]["dp_budget"]["epsilon"] + row["id"]

d = defaultdict(list)

for p in json_log_paths:
    with open(p) as f:
        run_dict = json.load(f)
    for t in run_dict["tasks"]:
        task_color = color_from_row(t)
        for block_id, block_budget in t["budget_per_block"].items():

            d["id"].append(t["id"])
            d["log_id"].append(np.log(t["id"] + 1))

            d["hashed_id"].append(hash(str(t["id"])) % 100)
            d["allocated"].append(t["allocated"])
            d["scheduler"].append(run_dict["simulator_config"]["scheduler_spec"]["method"])
            d["task_type"].append(task_color)
            d["total_blocks"].append(len(run_dict["blocks"]))
            d["n_blocks"].append(len(t["budget_per_block"]))

            d["block"].append(int(block_id))
            d["epsilon"].append(block_budget["dp_budget"]["epsilon"])
            d["block_selection"].append(run_dict["simulator_config"]["tasks_spec"]["curve_distributions"]["custom"]["read_block_selecting_policy_from_config"]["block_selecting_policy"])
            d["totalblocks_scheduler_selection"].append(f"{d['total_blocks'][-1]}-{d['scheduler'][-1]}-{d['block_selection'][-1]}")
            
            
df = pd.DataFrame(d).sort_values(["block", "id", "allocated"], ascending=[True, True, False])
        

In [ ]:
df.head()

In [ ]:
fig = px.bar(
#     df,
    df.query("allocated"),
    x="block",
    y="epsilon",
#     color="log_id",
    color="id",
    # barmode="group",
    pattern_shape="n_blocks",
    facet_col="totalblocks_scheduler_selection",
#     animation_frame="id"
)

fig

In [ ]:
dumps = load_latest_scheduling_results()


In [ ]:
px.histogram(dumps.query("total_blocks == 10 and scheduler == 'simplex' and block_selection == 'RandomBlocks' "), x="n_blocks")

In [ ]:
fig = px.bar(
    dumps.query("allocated and total_blocks == 10"),
#     dumps.query("total_blocks == 10"),    
    x="block",
    y="epsilon",
#     color="log_id",
    color="id",
    # barmode="group",
    pattern_shape="n_blocks",
    facet_col="totalblocks_scheduler_selection",
    facet_col_wrap=3,
    height=2000,
#     animation_frame="id"
)

fig

In [ ]:
df = load_latest_ray_experiment()

In [ ]:
px.line(df.sort_values('n_blocks'), x="n_blocks", y="n_allocated_tasks", color="scheduler", title="Multiblock DPF killer", facet_col="block_selecting_policy")